<a href="https://colab.research.google.com/github/Stonetenth2005/Machine-Learning/blob/main/9-ColumnTransformer/column_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [ ]:
df['fever'].isnull().sum()

np.int64(10)

Pre processing required -
*   OHE in gender
*   Ordinal encoding in cough
*   OHE in city
*   Simple imputing in fever






In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df.iloc[:,1:5], df.iloc[:,-1], test_size=0.3, random_state=0)

# Aam Zindagi (Without column transformer)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

In [ ]:
# Simple imputing for fever
si = SimpleImputer()
X_train_fever_transformed = si.fit_transform(X_train[['fever']])

X_test_fever_transformed = si.transform(X_test[['fever']])

In [ ]:
# Ordinal encoding for cough
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
X_train_cough_transformed = oe.fit_transform(X_train[['cough']])

X_test_cough_transformed = oe.transform(X_test[['cough']])

In [ ]:
# OHE for gender and city
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city_transformed = ohe.fit_transform(X_train[['gender', 'city']])

X_test_gender_city_transformed = ohe.transform(X_test[['gender', 'city']])

Now that all pre processing is done, we need to concatenate all these columns into one dataset

In [ ]:
# Extracting age
X_train_age = X_train.drop(columns=['gender', 'fever', 'cough', 'city']).values
X_test_age = X_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

In [ ]:
X_train_transformed = np.concatenate((X_train_age, X_train_gender_city_transformed, X_train_fever_transformed, X_train_cough_transformed), axis=1)
X_test_transformed = np.concatenate((X_test_age, X_test_gender_city_transformed, X_test_fever_transformed, X_test_cough_transformed), axis=1)

In [ ]:
pd.DataFrame(X_train_transformed)

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,102.0,1.0
1,0.0,0.0,0.0,1.0,99.0,0.0
2,0.0,1.0,0.0,0.0,99.0,1.0
3,0.0,0.0,1.0,0.0,104.0,1.0
4,0.0,0.0,1.0,0.0,100.0,1.0
...,...,...,...,...,...,...
65,0.0,0.0,1.0,0.0,101.0,1.0
66,1.0,0.0,0.0,0.0,99.0,0.0
67,1.0,0.0,0.0,1.0,104.0,0.0
68,0.0,0.0,0.0,0.0,104.0,0.0


# Mentos Zindagi (with column transformer)

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
transformer = ColumnTransformer(transformers = [
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories = [['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse_output=False, drop='first'), ['gender', 'city'])
], remainder='passthrough')

In [ ]:
X_test_transformed = transformer.fit_transform(X_test)
X_train_transformed = transformer.transform(X_train)